# 在v0p4基础上使用池文豪的标签数据

# 导包，设置根目录

In [1]:
import pandas as pd
import numpy as np
import os
import nltk
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import random
import pickle
import multiprocessing
import re

from collections import Counter
from itertools import chain
import datetime

# 更改工作目录为当前项目根目录
import sys
import os
os.chdir(sys.path[0])
print(os.getcwd())

C:\workfile\python\Log-diagnosis


C:\Users\jinca\anaconda3\envs\Log_diagnosis_python\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
C:\Users\jinca\anaconda3\envs\Log_diagnosis_python\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [10]:
word_df = pd.read_excel('pre_contest/dataset/word_label_df2.xlsx')
word_df.columns=['log','label']
word_df.groupby('label').size()

label
0    1476
1    3387
2    9320
3    2421
dtype: int64

In [11]:
word_df.head()

,log,label
0,'Management Subsys Health System_Health | Sens...,1
1,'Memory CPU1D0_DIMM_Stat | Correctable ECC | A...,2
2,'Memory DIMM050_Stat | Uncorrectable ECC | Ass...,1
3,'Memory DIMM050_Stat | Uncorrectable ECC | Ass...,2
4,'Drive Slot / Bay HDD_L_14_Status | Drive Faul...,3


In [12]:
log_str_list=list(word_df['log'])
label_list=list(word_df['label'])

# 分词及词频（仅保留分词后的字母数字下划线）统计，每条日志的词频向量

In [13]:
# 统计词频
word_lists = [word_tokenize(ith) for ith in log_str_list]
word_list=[re.sub(r'[^\w]','',str(xx)).lower() for xx in list(chain(*word_lists))]
word_frequency={}
for word in word_list:
    if word:
        if word in word_frequency:
            word_frequency[word]+=1
        else:
            word_frequency[word]=1
word_frequency=dict(sorted(word_frequency.items(),key=lambda item:item[1],reverse=True))
word_frequency_df=pd.DataFrame({'word':list(word_frequency.keys()),'frequency':list(word_frequency.values())})
# word_frequency_df.to_csv('pre_contest/dataset/word_frequency_df.txt',sep='\t')

In [17]:
# 计算每条日志的词频向量
frequency_vector_list = []
tag=0
for word in word_frequency:
    if tag%100==0:
        print(tag,datetime.datetime.now())
    frequency_vector = [sum(np.array([re.sub(r'[^\w]','',str(item)).lower() for item in word_lists[xth]]) == word) for xth in range(len(word_lists))]
    frequency_vector_list.append(frequency_vector)
    tag+=1


0 2022-03-13 22:24:20.812316
100 2022-03-13 22:28:15.515604
200 2022-03-13 22:32:09.978839
300 2022-03-13 22:36:04.592695
400 2022-03-13 22:39:59.211553
500 2022-03-13 22:43:53.740330
600 2022-03-13 22:47:48.631436
700 2022-03-13 22:51:43.316355
800 2022-03-13 22:55:38.454686
900 2022-03-13 22:59:33.278731
1000 2022-03-13 23:03:28.136807
1100 2022-03-13 23:07:23.087967
1200 2022-03-13 23:11:18.617653
1300 2022-03-13 23:15:13.624864
1400 2022-03-13 23:19:08.657098
1500 2022-03-13 23:23:03.551207
1600 2022-03-13 23:26:58.100002


In [ ]:
frequency_vector_df=pd.DataFrame(frequency_vector_list)
frequency_vector_df=frequency_vector_df.T
frequency_vector_df.columns=list(word_frequency.keys())
frequency_vector_df['label']=label_list
# frequency_vector_df['sn']=word_df['sn']
# frequency_vector_df['day']=word_df['day']
frequency_vector_df.to_csv('pre_contest/dataset/frequency_vector_df2.txt',sep='\t')

In [19]:
frequency_vector_df

,asserted,system,power,memory,initiated,drive,deasserted,boot,processor,state,...,dimm171_status,nic11,nic12,nic13,nic14,9120,dimm1b,memory_hot,00fc92f15f00,label
0,6,3,2,0,2,0,3,1,4,2,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,3,0,0,1,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,1
3,4,1,1,1,2,0,0,1,2,0,...,0,0,0,0,0,0,0,0,0,2
4,2,0,0,0,0,8,2,0,0,0,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16599,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
16600,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
16601,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
16602,2,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


# 训练XGB模型

In [20]:
random.seed(0)
feature=np.array(frequency_vector_list).T
label=np.array(label_list)
val_mask = [random.random() < 0.2 for _ in range(len(feature))]
train_mask = [not xx for xx in val_mask]
val_feature = feature[val_mask]
val_label = label[val_mask]
train_feature = feature[train_mask]
train_label = label[train_mask]

In [21]:
train_data=xgb.DMatrix(train_feature,label=train_label)
train_feature=xgb.DMatrix(train_feature)
val_feature=xgb.DMatrix(val_feature)
# xgb模型参数
params = {
    'booster':'gbtree',
    'objective':'multi:softmax',   # 多分类问题
    'num_class':4,  # 类别数，与multi softmax并用
    'gamma':0.1,    # 用于控制是否后剪枝的参数，越大越保守，一般0.1 0.2的样子
    'max_depth':6,  # 构建树的深度，越大越容易过拟合
    'lambda':2,  # 控制模型复杂度的权重值的L2 正则化项参数，参数越大，模型越不容易过拟合
    'subsample':1, # 随机采样训练样本
    'colsample_bytree':1,# 这个参数默认为1，是每个叶子里面h的和至少是多少
    # 对于正负样本不均衡时的0-1分类而言，假设h在0.01附近，min_child_weight为1
    #意味着叶子节点中最少需要包含100个样本。这个参数非常影响结果，
    # 控制叶子节点中二阶导的和的最小值，该参数值越小，越容易过拟合
    'silent':0,  # 设置成1 则没有运行信息输入，最好是设置成0
    'eta':0.3,  # 如同学习率
    'seed':1000,
    'nthread':12,  #CPU线程数
    #'eval_metric':'auc'
}
xgb_model=xgb.train(params,train_data,num_boost_round=500)

[23:30:34] WARNING: ..\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[23:30:34] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [22]:
train_pred=xgb_model.predict(train_feature)
val_pred=xgb_model.predict(val_feature)

In [23]:
# 指标评估
def macro_f1(label,prediction)  -> float:

    """
    计算得分
    :param target_df: [sn,fault_time,label]
    :param submit_df: [sn,fault_time,label]
    :return:
    """

    weights =  [3  /  7,  2  /  7,  1  /  7,  1  /  7]
    macro_F1 =  0.
    for i in  range(len(weights)):
        TP =  np.sum((label==i) & (prediction==i))
        FP =  np.sum((label!= i) & (prediction == i))
        FN =  np.sum((label == i) & (prediction!= i))
        precision = TP /  (TP + FP)  if  (TP + FP)  >  0  else  0
        recall = TP /  (TP + FN)  if  (TP + FN)  >  0  else  0
        F1 =  2  * precision * recall /  (precision + recall)  if  (precision + recall)  >  0  else  0
        macro_F1 += weights[i]  * F1
        
        print('Task %d:\n Prcesion %.2f, Recall %.2f, F1 %.2f' % (i+1, precision, recall, F1))
        
    return macro_F1

In [24]:
macro_f1(train_label,train_pred)

Task 1:
 Prcesion 0.93, Recall 0.44, F1 0.60
Task 2:
 Prcesion 0.79, Recall 0.90, F1 0.84
Task 3:
 Prcesion 0.95, Recall 0.97, F1 0.96
Task 4:
 Prcesion 0.91, Recall 0.94, F1 0.92


0.7656537429930985

In [25]:
macro_f1(val_label,val_pred)

Task 1:
 Prcesion 0.64, Recall 0.26, F1 0.37
Task 2:
 Prcesion 0.68, Recall 0.78, F1 0.73
Task 3:
 Prcesion 0.92, Recall 0.95, F1 0.94
Task 4:
 Prcesion 0.85, Recall 0.90, F1 0.88


0.6238448954277496

# 对初赛测试集进行预测

In [155]:
preliminary_submit_dataset=pd.read_csv('./pre_contest/dataset/preliminary_submit_dataset.txt',sep='\t')
test_log_str_list=list(preliminary_submit_dataset['log'])
test_word_lists = [word_tokenize(str(ith)) for ith in test_log_str_list]
test_frequency_vector_list = []
tag=0
for word in word_frequency:
    if tag%100==0:
        print(tag,datetime.datetime.now())
    test_frequency_vector = [sum(np.array([re.sub(r'[^\w]','',str(item)).lower() for item in test_word_lists[xth]]) == word) for xth in range(len(test_word_lists))]
    test_frequency_vector_list.append(test_frequency_vector)
    tag+=1


0 2022-03-13 21:25:46.391728
100 2022-03-13 21:26:06.530043
200 2022-03-13 21:26:26.661842
300 2022-03-13 21:26:46.802064
400 2022-03-13 21:27:06.947136
500 2022-03-13 21:27:27.096256
600 2022-03-13 21:27:47.252111
700 2022-03-13 21:28:07.397568
800 2022-03-13 21:28:27.558940
900 2022-03-13 21:28:47.717166
1000 2022-03-13 21:29:07.900085
1100 2022-03-13 21:29:28.086188
1200 2022-03-13 21:29:48.219670
1300 2022-03-13 21:30:08.368838
1400 2022-03-13 21:30:28.523133
1500 2022-03-13 21:30:48.686495
1600 2022-03-13 21:31:08.816415


In [167]:
test_frequency_vector_df=pd.DataFrame(test_frequency_vector_list)
test_frequency_vector_df=test_frequency_vector_df.T
test_frequency_vector_df.columns=list(word_frequency.keys())
test_frequency_vector_df[['sn','log','fault_time']]=preliminary_submit_dataset[['sn','log','fault_time']]
# test_frequency_vector_df.to_csv('pre_contest/dataset/test_frequency_vector_df.txt',sep='\t')

In [168]:
test_frequency_vector_df

,asserted,power,system,memory,drive,initiated,deasserted,processor,boot,state,...,nic11,dimm031_status,nic13,dimm171_status,9120,dimm1b,memory_hot,00fc92f15f00,sn,fault_time
0,2,1,2,0,0,4,0,0,2,0,...,0,0,0,0,0,0,0,0,000d33b21436,2020-09-02 16:42:54
1,4,2,3,1,0,2,0,0,1,2,...,0,0,0,0,0,0,0,0,005c5a9218ba,2020-06-28 19:05:16
2,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0079283bde6e,2020-04-26 21:32:44
3,5,2,3,1,0,2,0,0,1,2,...,0,0,0,0,0,0,0,0,007bdf23b62f,2020-06-16 18:40:39
4,6,0,3,2,0,1,0,0,3,0,...,0,0,0,0,0,0,0,0,00a577a8e54f,2020-04-07 07:16:55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3006,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,ffbf46b4af21,2019-12-28 20:10:01
3007,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,ffc229b6cd9a,2020-06-27 02:39:08
3008,2,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,ffd44698a52b,2020-01-21 15:46:56
3009,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,fff73a9e5bd5,2020-03-01 22:43:43


In [185]:
train_data=xgb.DMatrix(feature,label=label)
xgb_model_V1=xgb.train(params,train_data,num_boost_round=500)
test_feature_temp=np.array(test_frequency_vector_list).T
test_feature=xgb.DMatrix(test_feature_temp)
preliminary_submit_dataset['prediction']=xgb_model.predict(test_feature)

[21:57:21] WARNING: ..\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[21:57:21] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [209]:
preliminary_submit_dataset_a=preliminary_submit_dataset[['sn','fault_time','prediction']]
preliminary_submit_dataset_a['label']=preliminary_submit_dataset_a['prediction'].apply(lambda x : int(x))
preliminary_submit_dataset_a=preliminary_submit_dataset_a[['sn','fault_time','label']]
preliminary_submit_dataset_a.to_csv('pre_contest/dataset/preliminary_submit_dataset_a_V1.csv',sep=',',index=None)

C:\Users\jinca\AppData\Local\Temp\ipykernel_2032\2213533351.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preliminary_submit_dataset_a['label']=preliminary_submit_dataset_a['prediction'].apply(lambda x : int(x))


In [210]:
preliminary_submit_dataset_a

,sn,fault_time,label
0,000d33b21436,2020-09-02 16:42:54,3
1,005c5a9218ba,2020-06-28 19:05:16,2
2,0079283bde6e,2020-04-26 21:32:44,3
3,007bdf23b62f,2020-06-16 18:40:39,2
4,00a577a8e54f,2020-04-07 07:16:55,2
...,...,...,...
3006,ffbf46b4af21,2019-12-28 20:10:01,2
3007,ffc229b6cd9a,2020-06-27 02:39:08,2
3008,ffd44698a52b,2020-01-21 15:46:56,2
3009,fff73a9e5bd5,2020-03-01 22:43:43,2
